# Paper 7 – Deepfake Detection Benchmark


This notebook implements the **Paper 7** variant of the deepfake detector.



The code cells below typically follow this structure:

- Import libraries and define configuration values (paths, hyperparameters, device).
- Prepare datasets and data loaders for the benchmarks used in Paper 7.
- Implement the model architecture and training objective.
- Train and evaluate the model, printing metrics for later comparison with other papers.



> Run the cells from top to bottom to reproduce the results reported for Paper 7.

Paper link : https://arxiv.org/pdf/2403.07240 (2403.07240v1.pdf)

In [50]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as T

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    roc_curve,
    precision_score,
    recall_score,
    f1_score
)


In [51]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"

IMG_SIZE=224
BATCH_SIZE=16
EPOCHS=5
LR=2e-4

FFPP_REAL_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\FFPP_CViT\\train\\real"
FFPP_FAKE_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\FFPP_CViT\\train\\fake"

CELEBDF_REAL_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\CelebDF_images\\train\\real"
CELEBDF_FAKE_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\CelebDF_images\\train\\fake"

# Use DFDC validation split for faster cross-dataset evaluation
DFDC_REAL_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\DFDC\\validation\\real"
DFDC_FAKE_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\DFDC\\validation\\fake"


In [52]:
class ImageDataset(Dataset):

    def __init__(self,real_path,fake_path,jpeg_quality=None):

        self.samples=[]

        for f in os.listdir(real_path):
            self.samples.append((os.path.join(real_path,f),0))

        for f in os.listdir(fake_path):
            self.samples.append((os.path.join(fake_path,f),1))

        self.jpeg_quality=jpeg_quality

        self.tf=T.Compose([
            T.Resize((IMG_SIZE,IMG_SIZE)),
            T.ToTensor(),
            T.Normalize([0.5]*3,[0.5]*3)
        ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self,idx):

        path,label=self.samples[idx]
        img=Image.open(path).convert("RGB")

        if self.jpeg_quality:
            from io import BytesIO
            buf=BytesIO()
            img.save(buf,"JPEG",quality=self.jpeg_quality)
            img=Image.open(buf)

        img=self.tf(img)
        return img,label


In [38]:
def high_freq_image(x):

    fft=torch.fft.fft2(x,dim=(-2,-1))
    fft=torch.fft.fftshift(fft)

    B,C,H,W=fft.shape

    mask=torch.ones_like(fft)

    h=H//6
    w=W//6
    mask[:,:,H//2-h:H//2+h,W//2-w:W//2+w]=0

    fft_high=fft*mask

    fft_high=torch.fft.ifftshift(fft_high)
    img_high=torch.fft.ifft2(fft_high).real

    return img_high


In [39]:
class FrequencyConv(nn.Module):

    def __init__(self,channels):
        super().__init__()

        self.amp_conv=nn.Sequential(
            nn.Conv2d(channels,channels,3,padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU()
        )

        self.phase_conv=nn.Sequential(
            nn.Conv2d(channels,channels,3,padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU()
        )

    def forward(self,x):

        fft=torch.fft.fft2(x,dim=(-2,-1))

        amp=torch.abs(fft)
        phase=torch.angle(fft)

        amp=self.amp_conv(amp)
        phase=self.phase_conv(phase)

        fft_new=amp*torch.exp(1j*phase)

        out=torch.fft.ifft2(fft_new).real

        return out


In [40]:
class FreqNet(nn.Module):

    def __init__(self):
        super().__init__()

        # RGB stream
        self.rgb_conv1 = nn.Conv2d(3, 48, 3, padding=1)
        self.rgb_conv2 = nn.Conv2d(48, 96, 3, padding=1)

        # HF stream
        self.hf_conv1 = nn.Conv2d(3, 48, 3, padding=1)
        self.hf_conv2 = nn.Conv2d(48, 96, 3, padding=1)

        # single frequency interaction block (balanced)
        self.freq_block = FrequencyConv(96)

        # classifier
        self.conv3 = nn.Conv2d(96, 160, 3, padding=1)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(160, 2)

    def forward(self, x):

        # High-frequency branch from paper
        xh = high_freq_image(x)

        # RGB branch
        xr = F.relu(self.rgb_conv1(x))
        xr = F.max_pool2d(xr, 2)
        xr = F.relu(self.rgb_conv2(xr))

        # HF branch
        xf = F.relu(self.hf_conv1(xh))
        xf = F.max_pool2d(xf, 2)
        xf = F.relu(self.hf_conv2(xf))

        # discrepancy fusion (paper core idea)
        x = xr + xf

        # frequency learning
        x = self.freq_block(x)

        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))

        x = self.pool(x).flatten(1)

        logits = self.fc(x)
        return logits


In [41]:
model=FreqNet().to(DEVICE)

optimizer=torch.optim.AdamW(model.parameters(),lr=LR)
criterion=nn.CrossEntropyLoss()

train_loader=DataLoader(
    ImageDataset(FFPP_REAL_PATH,FFPP_FAKE_PATH),
    batch_size=BATCH_SIZE,
    shuffle=True
)


In [42]:
for epoch in range(EPOCHS):

    model.train()
    total_loss=0

    for imgs,labels in tqdm(train_loader):

        imgs=imgs.to(DEVICE)
        labels=labels.to(DEVICE)

        optimizer.zero_grad()

        logits=model(imgs)
        loss=criterion(logits,labels)

        loss.backward()
        optimizer.step()

        total_loss+=loss.item()

    print("Epoch",epoch+1,"Loss:",total_loss/len(train_loader))


100%|██████████| 2391/2391 [13:33<00:00,  2.94it/s]


Epoch 1 Loss: 0.5430835214295879


100%|██████████| 2391/2391 [12:57<00:00,  3.07it/s]


Epoch 2 Loss: 0.49005035466476743


100%|██████████| 2391/2391 [12:51<00:00,  3.10it/s]


Epoch 3 Loss: 0.45997577373426063


100%|██████████| 2391/2391 [12:54<00:00,  3.09it/s]


Epoch 4 Loss: 0.43362892815778564


100%|██████████| 2391/2391 [12:49<00:00,  3.11it/s]

Epoch 5 Loss: 0.40846394529369073


In [44]:
SAVE_DIR = "./checkpoints"
os.makedirs(SAVE_DIR, exist_ok=True)

MODEL_NAME = "paper7_model"
best_loss = float("inf")

def save_checkpoint(model, optimizer, epoch, loss):
    path = os.path.join(SAVE_DIR, f"{MODEL_NAME}_BEST.pth")
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": loss
    }, path)
    print("Saved BEST checkpoint:", path)

# Save checkpoint after training (uses last epoch's stats)
save_checkpoint(model, optimizer, epoch+1, total_loss/len(train_loader))

Saved BEST checkpoint: ./checkpoints\paper7_model_BEST.pth


In [53]:
# Load best saved model for evaluation
BEST_MODEL_PATH = "checkpoints/paper7_model_BEST.pth"

print("\nLoading best trained model from:", BEST_MODEL_PATH)

best_model = FreqNet().to(DEVICE)

state = torch.load(BEST_MODEL_PATH, map_location=DEVICE)
best_model.load_state_dict(state["model_state_dict"])

best_model.eval()
print("✔ Best model loaded successfully")


Loading best trained model from: checkpoints/paper7_model_BEST.pth
✔ Best model loaded successfully


C:\Users\vk200\AppData\Local\Temp\ipykernel_25252\3275649164.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(BEST_MODEL_PATH, map_location=DEVICE)


In [54]:
@torch.no_grad()
def evaluate_model(loader, model):

    model.eval()

    all_probs=[]
    all_labels=[]
    all_preds=[]

    correct=0
    total=0

    # show progress so long runs (e.g., DFDC) are visible
    for imgs,labels in tqdm(loader, desc="Evaluating", leave=False):

        imgs=imgs.to(DEVICE)
        labels=labels.to(DEVICE)

        logits=model(imgs)

        probs=torch.softmax(logits,1)[:,1]
        preds=logits.argmax(1)

        correct+=(preds==labels).sum().item()
        total+=labels.size(0)

        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    acc=correct/total
    auc=roc_auc_score(all_labels,all_probs)
    ap=average_precision_score(all_labels,all_probs)

    precision=precision_score(all_labels,all_preds, zero_division=0)
    recall=recall_score(all_labels,all_preds, zero_division=0)
    f1=f1_score(all_labels,all_preds, zero_division=0)

    fpr,tpr,_=roc_curve(all_labels,all_probs)
    fnr=1-tpr
    eer=fpr[np.nanargmin(np.abs(fnr-fpr))]

    return {
        "ACC":acc,
        "AUC":auc,
        "Precision":precision,
        "Recall":recall,
        "F1":f1,
        "AP":ap,
        "EER":eer
    }

In [47]:
print("\n===== FF++ Evaluation (TEST SET) =====")

# Use FF++ test split for evaluation
FFPP_REAL_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\FFPP_CViT\\test\\real"
FFPP_FAKE_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\FFPP_CViT\\test\\fake"

ffpp_loader = DataLoader(
    ImageDataset(FFPP_REAL_PATH,FFPP_FAKE_PATH),
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(evaluate_model(ffpp_loader, best_model))


===== FF++ Evaluation (TEST SET) =====
{'ACC': 0.7210537967677663, 'AUC': 0.6729731399620718, 'Precision': 0.8599609375, 'Recall': 0.7896341463414634, 'F1': 0.8232984293193717, 'AP': 0.9074927604393297, 'EER': np.float64(0.38682784493538974)}


In [55]:
print("\n===== Cross Dataset Evaluation =====")

celeb_loader = DataLoader(
    ImageDataset(CELEBDF_REAL_PATH,CELEBDF_FAKE_PATH),
    batch_size=BATCH_SIZE
)

dfdc_loader = DataLoader(
    ImageDataset(DFDC_REAL_PATH,DFDC_FAKE_PATH),
    batch_size=BATCH_SIZE
)

print("CelebDF:", evaluate_model(celeb_loader, best_model))
print("DFDC:", evaluate_model(dfdc_loader, best_model))


===== Cross Dataset Evaluation =====


CelebDF: {'ACC': 0.8475969120942973, 'AUC': 0.5442453827286273, 'Precision': 0.9053987122337791, 'Recall': 0.9275335578209327, 'F1': 0.9163324826831936, 'AP': 0.90161186937273, 'EER': np.float64(0.46335403726708074)}


DFDC: {'ACC': 0.7964213807884652, 'AUC': 0.5130245404831624, 'Precision': 0.8029151654110711, 'Recall': 0.989824348879467, 'F1': 0.8866262772402568, 'AP': 0.8267667544993549, 'EER': np.float64(0.4997512025211478)}


In [56]:
print("\n===== JPEG Robustness (FF++ TEST) =====")

# Ensure FF++ test split here
FFPP_REAL_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\FFPP_CViT\\test\\real"
FFPP_FAKE_PATH = r"C:\\Users\\vk200\\OneDrive\\Desktop\\Benchmarking\\FFPP_CViT\\test\\fake"

for q in [90,70,50,30]:

    jpeg_loader = DataLoader(
        ImageDataset(FFPP_REAL_PATH,FFPP_FAKE_PATH,jpeg_quality=q),
        batch_size=BATCH_SIZE
)

    print(f"JPEG {q}:", evaluate_model(jpeg_loader, best_model))


===== JPEG Robustness (FF++ TEST) =====


JPEG 90: {'ACC': 0.6894694118515239, 'AUC': 0.6686080716917558, 'Precision': 0.8624973898517436, 'Recall': 0.7407639885222381, 'F1': 0.797009165460685, 'AP': 0.9060709837566194, 'EER': np.float64(0.3918299291371405)}


JPEG 70: {'ACC': 0.7379529186037931, 'AUC': 0.6806739028636394, 'Precision': 0.8590458195559755, 'Recall': 0.8153694404591105, 'F1': 0.8366379905230713, 'AP': 0.9092140663680772, 'EER': np.float64(0.3826594414339308)}


JPEG 50: {'ACC': 0.7458490148328537, 'AUC': 0.6826325329540106, 'Precision': 0.8597162591002426, 'Recall': 0.8259505021520803, 'F1': 0.84249519802433, 'AP': 0.9101240583592081, 'EER': np.float64(0.3772405168820342)}


JPEG 30: {'ACC': 0.7362556268910043, 'AUC': 0.6802822740285736, 'Precision': 0.8580608580608581, 'Recall': 0.8142037302725968, 'F1': 0.8355571914971933, 'AP': 0.9093169556080796, 'EER': np.float64(0.3776573572321801)}
